In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import cv2

In [ ]:
def watch_batch(fn,batch,fps):
    print("watch_batch")
    #sec = np.shape(batch)/fps

    # Syntax: VideoWriter_fourcc(c1, c2, c3, c4) # Concatenates 4 chars to a fourcc code
    #  cv2.VideoWriter_fourcc('M','J','P','G') or cv2.VideoWriter_fourcc(*'MJPG)
    
    fourcc = cv2.VideoWriter_fourcc(*'mp4v') # FourCC is a 4-byte code used to specify the video codec.
    # A video codec is software or hardware that compresses and decompresses digital video. 
    # In the context of video compression, codec is a portmanteau of encoder and decoder, 
    # while a device that only compresses is typically called an encoder, and one that only 
    # decompresses is a decoder. Source - Wikipedia
    
    #Syntax: cv2.VideoWriter( filename, fourcc, fps, frameSize )
    fn = fn.replace('.mp4','')
    print(rslt_path+'/1V/best_batch/'+fn+'BB.mp4')
    video = cv2.VideoWriter(rslt_path+'/1V/best_batch/'+fn+'BB.mp4', fourcc, float(fps), (target_width, target_height))
    
    
    
    for frame in range(batch.shape[0]):
        batch[frame].convertTo(image, np.uint8);
        #batch[frame] = batch[frame].astype(np.uint8)
        video.write(image)
    
    video.release()

In [ ]:
batch_type = 1
batch_no=0
target_height = 120
target_width = 160
frame_max = 4000

base_vigia_dir = "/media/jtstudents/HDD/.zuble/vigia"
rslt_path = base_vigia_dir+'/zhen++/parameters_results'

file_name = '/media/jtstudents/HDD/.zuble/xdviol/test/v=-fOWSLV6Esw__#1_label_B4-0-0.mp4'
#file_name = '/raid/DATASETS/anomaly/UCF_Crimes/Videos/Training_Normal_Videos_Anomaly/Normal_Videos308_x264.mp4'
video = cv2.VideoCapture(file_name)
total_frame = video.get(cv2.CAP_PROP_FRAME_COUNT)
fps = video.get(cv2.CAP_PROP_FPS)
#mtcnn_detector = mtcnn.mtcnn.MTCNN()
divid_no = 1

if total_frame > frame_max:
    total_frame_int = int(total_frame)
    if total_frame_int % frame_max == 0:
        divid_no = int(total_frame / frame_max)
    else:
        divid_no = int(total_frame / frame_max) + 1

passby = 0
#updates the start frame to 0,4000,8000... excluding the last batch
if batch_no != divid_no - 1:
    while video.isOpened and passby < frame_max * batch_no:
        passby += 1
        success, image = video.read()
        if success == False:
            break
#updates the last batch starting frame 
else:
    if batch_type==1:
        #print("1")
        while video.isOpened and passby < total_frame - frame_max:
            passby += 1
            success, image = video.read()
            if success == False:
                break
    #last batch must have >= 400 otherwise it falls back to batch_type 1
    if batch_type==2 and total_frame - (frame_max * batch_no) >= frame_max*0.1:
        #print("2")
        while video.isOpened and passby < frame_max * batch_no:
            passby += 1
            success, image = video.read()
            if success == False:
                break
    else:
        while video.isOpened and passby < total_frame - frame_max:
            passby += 1
            success, image = video.read()
            if success == False:
                break

        
batch_frames = [] 
batch_imgs = [] 
counter = 0

while video.isOpened:
    
    success, image = video.read()
    if success == False:
        break
    
    #print(np.shape(image)) 
    
    image = cv2.resize(image, (target_width, target_height))
    image_array = np.array(image)/255.0 #normalize
    batch_frames.append(image_array)
    batch_imgs.append(image)
    cv2.imshow('frame', image)
    
    counter += 1
    if counter > frame_max:
        break
        
video.release()
batch_frames = np.array(batch_frames)

print(batch_frames[3].shape)

print("\t-batch",batch_no,"[",passby,", ... ] ",batch_frames.shape)    


In [ ]:
watch_batch("v=-fOWSLV6Esw__#1_label_B4-0-0",batch_frames,24)

In [ ]:
import numpy as np
import cv2

def quit_key_action(**params):
    global is_quit
    is_quit = True
def rewind_key_action(**params):
    global frame_counter
    frame_counter = max(0, int(frame_counter - (fps * 5)))
    video.set(cv2.CAP_PROP_POS_FRAMES, frame_counter)
def forward_key_action(**params):
    global frame_counter
    frame_counter = min(int(frame_counter + (fps * 5)), total_frame - 1)
    video.set(cv2.CAP_PROP_POS_FRAMES, frame_counter)
def pause_key_action(**params):
    global is_paused
    is_paused = not is_paused
# Map keys to buttons
key_action_dict = {
    ord('q'): quit_key_action,
    ord('a'): rewind_key_action,
    ord('d'): forward_key_action,
    ord('s'): pause_key_action,
    ord(' '): pause_key_action
}
def key_action(_key):
    if _key in key_action_dict:
        key_action_dict[_key]()


base_vigia_dir = "/media/jtstudents/HDD/.zuble/vigia"
rslt_path = base_vigia_dir+'/zhen++/parameters_results'
        
batch_type = 1
batch_no=0
target_height = 120
target_width = 160
frame_max = 4000

#input
file_name = '/media/jtstudents/HDD/.zuble/xdviol/test/v=-fOWSLV6Esw__#1_label_B4-0-0.mp4'
video = cv2.VideoCapture(file_name)
total_frame = video.get(cv2.CAP_PROP_FRAME_COUNT)
fps = video.get(cv2.CAP_PROP_FPS)

#output
final_path = rslt_path+'/1V/best_batch/test_BB.mp4'
out = cv2.VideoWriter(final_path, cv2.VideoWriter_fourcc(*'mp4v'), int(fps), (target_width,target_height), False)

counter = 0
batch_frames = [] 
while video.isOpened:
    
    success, image = video.read()
    if success == False:
        break
    
    print("image_shape",np.shape(image)) 
    
    image = cv2.resize(image, (target_width, target_height))
    image_array = np.array(image)/255.0 #normalize
    batch_frames.append(image_array)

    #cv2.putText(image, strftime("%H:%M:%S"), (10,70), cv2.FONT_HERSHEY_SIMPLEX, 2,(0,255,0),2,cv2.LINE_AA)
    
    cv2.imshow('frame', image)
    # Wait for any key press and pass it to the key action
    key = cv2.waitKey(1)
    key_action(key)
    
    counter += 1
    if counter > frame_max:
        break
    
print("image_shape",np.shape(batch_frames)) 
for i in range(len(batch_frames)):
    print("frame",i)
    data = batch_frames[i].astype('uint8') * 255
    data = cv2.cvtColor(data, cv2.COLOR_RGB2BGR)
    out.write(data)
    
out.release()
